In [4]:
import pymongo

In [3]:
#!pip install pymongo==4.3.3

     |████████████████████████████████| 501 kB 1.7 MB/s            
     |████████████████████████████████| 283 kB 1.5 MB/s            
You should consider upgrading via the '/home/train/venvspark/bin/python -m pip install --upgrade pip' command.


In [5]:
# mongodb bağlantı
myclient = pymongo.MongoClient(host="mongodb://localhost:27017/?authSource=test", 
                               username='myTester', password='Ankara06')

In [6]:
# veri tabanlarını listele
print(myclient.list_database_names())

['test']


In [7]:
# bir veri tabanı seç veya yoksa oluştur.
# Not: İçine birşey konmadıkçamongodb'de veri tabanı gerçekten oluşmaz.
db = myclient["test"]

In [8]:
# Bir collection oluştur.
dataops_collection = db["dataops_collection"]

In [9]:
db.list_collection_names()

['customers', 'churn']

In [10]:
# pandas Dataframe oku/oluştur
import pandas as pd
df = pd.read_csv("/home/train/datasets/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [12]:
# pandas dataframe'i json formatına çevir
import json
records = json.loads(df.T.to_json()).values()

In [13]:
type(records)

dict_values

In [14]:
#json formatına çevrilen dataframe'i mongodb'ye yaz
x = dataops_collection.insert_many(records)

In [21]:
y = dataops_collection.find_one()

In [22]:
print(y)

{'_id': ObjectId('6421ae3e80aadbc60c181a7c'), 'RowNumber': 1, 'CustomerId': 15634602, 'Surname': 'Hargrave', 'CreditScore': 619, 'Geography': 'France', 'Gender': 'Female', 'Age': 42, 'Tenure': 2, 'Balance': 0.0, 'NumOfProducts': 1, 'HasCrCard': 1, 'IsActiveMember': 1, 'EstimatedSalary': 101348.88, 'Exited': 1}


# From MongoDB to pandas

In [23]:
x_all = dataops_collection.find()

In [24]:
pd_df = pd.DataFrame(x_all)

In [25]:
pd_df.head()

,_id,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,6421ae3e80aadbc60c181a7c,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,6421ae3e80aadbc60c181a7d,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,6421ae3e80aadbc60c181a7e,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,6421ae3e80aadbc60c181a7f,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,6421ae3e80aadbc60c181a80,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [26]:
pd_df.count()

_id                10000
RowNumber          10000
CustomerId         10000
Surname            10000
CreditScore        10000
Geography          10000
Gender             10000
Age                10000
Tenure             10000
Balance            10000
NumOfProducts      10000
HasCrCard          10000
IsActiveMember     10000
EstimatedSalary    10000
Exited             10000
dtype: int64